# EXTRAÇÃO E ANÁLISE DE DADOS DO ENEM 2019

In [1]:
# Importar as biblioteca necessárias
import pandas as pd
import numpy as np
from io import BytesIO
import zipfile
import requests
import os

In [ ]:
# Criar um diretóro para armazenar o conteúdo de extração dos dados do ENEM
os.makedirs('./Projeto_enem2019', exist_ok=True)

In [ ]:
# Define URL para download e extração do arquivo microdados_enem_2019.zip
url = "https://download.inep.gov.br/microdados/microdados_enem_2019.zip"

# Faz o Download do conteúdo
filebytes = BytesIO(
    requests.get(url).content
)

# Extrair o conteúdo do zipfile
myzip = zipfile.ZipFile(filebytes)
myzip.extractall("./Projeto_enem2019")

### O arquivo MICRODADOS_ENEM_2019.csv é muito grande, cerca de 3GB e ler tudo isso de uma vez na memória, é preciso extrair os dados por partes e depois concátena-los só com os dados do estado de MG, conforme solicitado pela atividade.

In [ ]:
# Divide arquivo em mais partes por questões de memória filtrando por estado

os.makedirs('./Projeto_enem2019/CHUNK/', exist_ok=True)

chunk_size=1000000
batch_no=1
for chunk in pd.read_csv('./Projeto_enem2019/DADOS/MICRODADOS_ENEM_2019.csv', sep = ';', decimal = ',', chunksize=chunk_size, encoding = "Latin1"):
    chunk[chunk.SG_UF_RESIDENCIA == 'MG'].to_csv('./Projeto_enem2019/CHUNK/chunk'+str(batch_no)+'.csv', sep = ';', decimal = ',', encoding = "Latin1", index=False)
    batch_no+=1

# Cria um DataFrame único com todos os arquivos filtrados
columns= chunk.columns
enem = pd.DataFrame(columns = columns)
for i in range(1, batch_no):
    input = pd.read_csv('./Projeto_enem2019/CHUNK/chunk'+str(i)+'.csv', sep = ';', decimal = ',', encoding = "Latin1")
    enem = enem.append(input, ignore_index =True)

# Cria um arquivo CSV único com os dados filtrados 
enem.to_csv('./Projeto_enem2019/CHUNK/MICRODADOS_ENEM_2019_MG.csv', sep = ';', decimal = ',', encoding = "Latin1", index=False)

In [2]:
# Carrega o dataframe caso já tenha feito o processo de separação dos dados de MG
enem = pd.read_csv('./Projeto_enem2019/CHUNK/MICRODADOS_ENEM_2019_MG.csv', sep = ';', decimal = ',', encoding = "Latin1")

In [3]:
# Verificando as 5 primeiros linha do arquivo MICRODADOS_ENEM_2019_MG
enem.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001005671,2019,3170206,Uberlândia,31,MG,21,M,1,1,...,B,A,A,D,A,A,D,B,B,B
1,190001009861,2019,3111705,Canaã,31,MG,20,M,1,1,...,A,A,A,B,A,A,C,A,A,A
2,190001021622,2019,3104007,Araxá,31,MG,24,M,1,2,...,A,A,A,C,B,A,C,A,A,B
3,190001026636,2019,3106200,Belo Horizonte,31,MG,33,M,1,1,...,B,A,B,B,A,A,D,A,D,B
4,190001026742,2019,3162401,São João da Ponte,31,MG,23,M,1,3,...,A,A,A,B,B,A,C,A,B,A


In [4]:
# Verificando os tipos de dados da tabela
dict(enem.dtypes)

{'NU_INSCRICAO': dtype('int64'),
 'NU_ANO': dtype('int64'),
 'CO_MUNICIPIO_RESIDENCIA': dtype('int64'),
 'NO_MUNICIPIO_RESIDENCIA': dtype('O'),
 'CO_UF_RESIDENCIA': dtype('int64'),
 'SG_UF_RESIDENCIA': dtype('O'),
 'NU_IDADE': dtype('int64'),
 'TP_SEXO': dtype('O'),
 'TP_ESTADO_CIVIL': dtype('int64'),
 'TP_COR_RACA': dtype('int64'),
 'TP_NACIONALIDADE': dtype('int64'),
 'CO_MUNICIPIO_NASCIMENTO': dtype('float64'),
 'NO_MUNICIPIO_NASCIMENTO': dtype('O'),
 'CO_UF_NASCIMENTO': dtype('float64'),
 'SG_UF_NASCIMENTO': dtype('O'),
 'TP_ST_CONCLUSAO': dtype('int64'),
 'TP_ANO_CONCLUIU': dtype('int64'),
 'TP_ESCOLA': dtype('int64'),
 'TP_ENSINO': dtype('float64'),
 'IN_TREINEIRO': dtype('int64'),
 'CO_ESCOLA': dtype('float64'),
 'CO_MUNICIPIO_ESC': dtype('float64'),
 'NO_MUNICIPIO_ESC': dtype('O'),
 'CO_UF_ESC': dtype('float64'),
 'SG_UF_ESC': dtype('O'),
 'TP_DEPENDENCIA_ADM_ESC': dtype('float64'),
 'TP_LOCALIZACAO_ESC': dtype('float64'),
 'TP_SIT_FUNC_ESC': dtype('float64'),
 'IN_BAIXA_VISAO'

In [5]:
# Gerando estatísticas descritivas.
enem.SG_UF_RESIDENCIA.describe()

count     538896
unique         1
top           MG
freq      538896
Name: SG_UF_RESIDENCIA, dtype: object

In [6]:
# Contando o número de nulos
enem.SG_UF_RESIDENCIA.isnull().sum()

0

In [7]:
# Contato o número de linhas e colunas
enem.shape

(538896, 136)

In [8]:
# Quantidade relativa de nulos
enem.SG_UF_RESIDENCIA.isnull().sum()/enem.shape[0]

0.0

### Quetionário da atividade prática

In [9]:
# 1-Qual é a média da nota em matemática de todos os alunos mineiros?
enem.NU_NOTA_MT.astype(float).mean()

546.7962076512125

In [10]:
# 2-Qual é a média da nota em Linguagens e Códigos de todos os alunos mineiros?
enem.NU_NOTA_LC.astype(float).mean()

531.2155500475544

In [11]:
# 3-Qual é a média da nota em Ciências Humanas dos alunos do sexo FEMININO mineiros?
enem.loc[
    enem.TP_SEXO =="F",
    "NU_NOTA_CH"
].astype(float).mean()

515.1270983575349

In [12]:
# 4-Qual é a média da nota em Ciências Humanas dos alunos do sexo MASCULINO?
enem.loc[
    enem.TP_SEXO =="M",
    "NU_NOTA_CH"
].astype(float).mean()

529.6982704731574

In [13]:
# 5-Qual é a média da nota em Matemática dos alunos do sexo FEMININO que moram na cidade de Montes Claros?
enem.loc[
    (enem.NO_MUNICIPIO_RESIDENCIA =="Montes Claros") &
    (enem.TP_SEXO =="F"),
    "NU_NOTA_MT"
].astype(float).mean()

525.4776724249901

In [14]:
# 6-Qual é a média da nota em Matemática dos alunos do município de Sabará que possuem TV por assinatura na residência?
enem.loc[
    (enem.NO_MUNICIPIO_RESIDENCIA =="Sabará") &
    (enem.Q021 == "B"),
    "NU_NOTA_MT"
].astype(float).mean()

543.2927556818183

In [15]:
# 7-Qual é a média da nota em Ciências Humanas dos alunos mineiros que possuem dois fornos micro-ondas em casa?
enem.loc[
    (enem.Q016 == "C"),
    "NU_NOTA_CH"
].astype(float).mean()

557.2765986394558

In [16]:
# 8-Qual é a nota média em Matemática dos alunos mineiros cuja mãe completou a pós-graduação?
enem.loc[
    (enem.Q002 == "G"),
    "NU_NOTA_MT"
].astype(float).mean()

620.007062070985

In [17]:
# 9-Qual é a nota média em Matemática dos alunos de Belo Horizonte e de Conselheiro Lafaiete?
enem.loc[
    (enem.NO_MUNICIPIO_RESIDENCIA.isin(['Belo Horizonte','Conselheiro Lafaiete'])),
    "NU_NOTA_MT"
].astype(float).mean()

578.0392265100068

In [18]:
# 10-Qual é a nota média em Ciências Humanas dos alunos mineiros que moram sozinhos?
enem.loc[
    (enem.Q005 == 1),
    "NU_NOTA_CH"
].astype(float).mean()

534.4573388609205

In [19]:
# 11-Qual é a nota média em Ciências Humanas dos alunos mineiros cujo pai completou pós-graduação e possuem renda familiar entre R$ 8.982,01 e R$ 9.980,00.
enem.loc[
    (enem.Q001 == "G") &
    (enem.Q006 == "M"),
    "NU_NOTA_CH"
].astype(float).mean()

586.7231663685159

In [20]:
# 12-Qual é a nota média em Matemática dos alunos do sexo Feminino que moram em Lavras e escolheram “Espanhol” como língua estrangeira?
enem.loc[
    (enem.TP_SEXO =="F") &
    (enem.NO_MUNICIPIO_RESIDENCIA.isin(['Lavras'])) &
    (enem.TP_LINGUA ==1),
    "NU_NOTA_MT"
].astype(float).mean()

510.80950782997775

In [21]:
# 13-Qual é a nota média em Matemática dos alunos do sexo Masculino que moram em Ouro Preto?
enem.loc[
    (enem.TP_SEXO =="M") &
    (enem.NO_MUNICIPIO_RESIDENCIA.isin(['Ouro Preto'])),
    "NU_NOTA_MT"
].astype(float).mean()

555.0832520325198

In [22]:
# 14-Qual é a nota média em Ciências Humanas dos alunos surdos?
enem.loc[
    (enem.IN_SURDEZ ==1),
    "NU_NOTA_CH"
].astype(float).mean()

435.38796296296283

In [23]:
# 15-Qual é a nota média em Matemática dos alunos do sexo FEMININO, que moram em Belo Horizonte, Sabará, Nova Lima e Betim e possuem dislexia?
enem.loc[
    (enem.TP_SEXO =="F") &
    (enem.NO_MUNICIPIO_RESIDENCIA.isin(['Belo Horizonte','Sabará','Nova Lima','Betim'])) &
    (enem.IN_DISLEXIA ==1),
    "NU_NOTA_MT"
].astype(float).mean()

582.1935483870968